# France Stduy part I: estimation of forced responses


This notebook provides code for estimation of the forced responses (i.e., ALL, NAT, GHG), both at the global and regional scales, for each CMIP6 model considered.

## 1. Introduction

As **input**, we use pre-processed CMIP6 model outputs: 
- Xg: GSAT annual mean temperature. An array [year,experiment,model].
- Xf: Annual mean temperature over France. Pre-processing in this case includes intrepolation on a .1° grid (accouting for the land-sea mask), masking of Mainland France, computation of the spatial average over France, then the annual mean temperature. An array [year,experiment,model].
- Xf_jja and Xf_djf: same over summer / winter. Array [year,experiment,model].

As **output**, we provide:
- Xg_fit: the estimated forced GSAT responses. An array [year,forcing,model]. Forcing is one of "all","nat","ghg", given other terms (e.g., "ant","oa") can be recovered assuming additivity.
- Xf_fit: same for the regional temperature.
- Xfjja_fit and Xfdjf_fit: same for the regional and seasonal temperature. The "ghg" response is not estimated in these cases, as it is not used in our analysis.

Note that inputs (e.g., Xg) include the data for various SSP scenarios (1-1.9, 2-4.5, 3-7.0, 5-8.5), but the estimation of forced responses is made for "histssp245" only in this notebook. Replication to other scenarios is easy.

-------------------------------------------------------------------------------------------------------------------

## 2. Get started

We start with some preliminary R commands.

Note that the R package KCC has to be installed first. See instructions on https://gitlab.com/saidqasmi/KCC.

In [1]:
library(KCC)
library(gam)
library(abind)
library(MASS)
# Set random number generator to ensure reproducibility
set.seed(13)

Loading required package: splines

Loading required package: foreach

Loaded gam 1.20




As the estimation of the ANT response involves smoothing splines, we need to select the number of equivalent degrees of freedom.

In [2]:
experiments = c("histssp245", "1pctCO2", "histghg")
nexp = length(experiments)
# The effective df in x_fit (ANT component)
dfg = c( 6, 5, 6 )
    names(dfg) = experiments
dff = c( 8, 5, 6 )
    names(dff) = experiments

Load input data (Xg, Xf, Xfjja, Xfdjf)

In [3]:
load("Processed_CMIP6_data.Rdata")
year = as.numeric(dimnames(Xg)$year)
Models = dimnames(Xg)$model
ny = length(year)
Nmod = length(Models)

Initialize outputs: Xg_fit, Xf_fit, Xfjja_fit, Xfdjf_fit

In [4]:
Xg_fit = array(NA,dim=c(ny,3,Nmod),#
               dimnames=list(year = as.character(year),#
                             forc = c("all","nat","ghg"),#
                             model = Models))

Xf_fit = array(NA,dim=c(ny,3,Nmod),#
               dimnames=list(year = as.character(year),#
                             forc = c("all","nat","ghg"),#
                             model = Models))

Xfjja_fit = array(NA,dim=c(ny,3,Nmod),#
               dimnames=list(year = as.character(year),#
                             forc = c("all","nat","ghg"),#
                             model = Models))

Xfdjf_fit = array(NA,dim=c(ny,3,Nmod),#
               dimnames=list(year = as.character(year),#
                             forc = c("all","nat","ghg"),#
                             model = Models))

## 3. Estimating ALL and NAT responses

We first derive an simple EBM response to the CMIP6 NAT forcings time-series.

In [5]:
load("FF_CMIP6_new.rda")
load("ebm_params.rda")
Enat = ebm_response(FF,ebm_params,year,Nres=1)

ALL and NAT responses in GSAT (i.e., Xg --> Xg_fit).

In [7]:
message("Global T: Xg_fit")

# General case
Models_scen_full = Models[ apply(is.na(Xg[,"histssp245",]),2,sum)==0 ]
Xg_fit[,c("all","nat"),Models_scen_full] = x_fit(Xg[,"histssp245",Models_scen_full],Enat,dfg["histssp245"],ant=F)
# Particular cases
# CAMS-CSM1-0 (simulation stops in 2099)
    year_cams = as.character(1850:2099)
    Xg_fit[year_cams,c("all","nat"),"CAMS-CSM1-0"] = x_fit( Xg[year_cams,"histssp245",c("CAMS-CSM1-0","CAMS-CSM1-0")], Enat[year_cams,],dfg["histssp245"],ant=F)[,,1]
    Xg_fit["2100",,"CAMS-CSM1-0"] = 2 * Xg_fit["2099",,"CAMS-CSM1-0"] - Xg_fit["2098",,"CAMS-CSM1-0"]


Global T: Xg_fit

		ACCESS-CM2

		ACCESS-ESM1-5

		AWI-CM-1-1-MR

		CanESM5-CanOE

		CanESM5

		CESM2

		CESM2-WACCM

		CMCC-CM2-SR5

		CNRM-CM6-1-HR

		CNRM-CM6-1

		CNRM-ESM2-1

		EC-Earth3-Veg

		FGOALS-f3-L

		FGOALS-g3

		GISS-E2-1-G

		INM-CM4-8

		IPSL-CM6A-LR

		MIROC6

		MIROC-ES2L

		MPI-ESM1-2-HR

		MPI-ESM1-2-LR

		MRI-ESM2-0

		NorESM2-LM

		NorESM2-MM

		TaiESM1

		UKESM1-0-LL

		CAMS-CSM1-0

		CAMS-CSM1-0



Same thing for the France temperature Xf, then seasonal France temperature...

In [9]:
message("France T: Xf_fit")

# General case
Xf_fit[,c("all","nat"),Models_scen_full] = x_fit(Xf[,"histssp245",Models_scen_full],Enat,dff["histssp245"],ant=F)
# Particular cases
# CAMS-CSM1-0 (simulation stops in 2099)
    year_cams = as.character(1850:2099)
    Xf_fit[year_cams,c("all","nat"),"CAMS-CSM1-0"] = x_fit(Xf[year_cams,"histssp245",c("CAMS-CSM1-0","CAMS-CSM1-0")], Enat[year_cams,],dff["histssp245"],ant=F)[,,1]
    Xf_fit["2100",,"CAMS-CSM1-0"] = 2 * Xf_fit["2099",,"CAMS-CSM1-0"] - Xf_fit["2098",,"CAMS-CSM1-0"]

France T: Xf_fit

		ACCESS-CM2

		ACCESS-ESM1-5

		AWI-CM-1-1-MR

		CanESM5-CanOE

		CanESM5

		CESM2

		CESM2-WACCM

		CMCC-CM2-SR5

		CNRM-CM6-1-HR

		CNRM-CM6-1

		CNRM-ESM2-1

		EC-Earth3-Veg

		FGOALS-f3-L

		FGOALS-g3

		GISS-E2-1-G

		INM-CM4-8

		IPSL-CM6A-LR

		MIROC6

		MIROC-ES2L

		MPI-ESM1-2-HR

		MPI-ESM1-2-LR

		MRI-ESM2-0

		NorESM2-LM

		NorESM2-MM

		TaiESM1

		UKESM1-0-LL

		CAMS-CSM1-0

		CAMS-CSM1-0



In [10]:
message("	France T, summer and winter: Xfjja_fit, Xfdjf_fit")

# General case
Xfjja_fit[,c("all","nat"),Models_scen_full] = x_fit(Xfjja[,"histssp245",Models_scen_full],Enat,dff["histssp245"],ant=F)
Xfdjf_fit[2:ny,c("all","nat"),Models_scen_full] = x_fit(Xfdjf[2:ny,"histssp245",Models_scen_full],Enat[2:ny,],dff["histssp245"],ant=F)
# Particular cases
# CAMS-CSM1-0 (simulation stops in 2099)
	year_cams = as.character(1850:2099)
	Xfjja_fit[year_cams,c("all","nat"),"CAMS-CSM1-0"] = x_fit(Xfjja[year_cams,"histssp245",c("CAMS-CSM1-0","CAMS-CSM1-0")], Enat[year_cams,],dff["histssp245"],ant=F)[,,1]
	Xfjja_fit["2100",,"CAMS-CSM1-0"] = 2 * Xfjja_fit["2099",,"CAMS-CSM1-0"] - Xfjja_fit["2098",,"CAMS-CSM1-0"]
	Xfdjf_fit[year_cams[2:(ny-1)],c("all","nat"),"CAMS-CSM1-0"] = x_fit(Xfdjf[year_cams[2:(ny-1)],"histssp245",c("CAMS-CSM1-0","CAMS-CSM1-0")], Enat[year_cams[2:(ny-1)],],dff["histssp245"],ant=F)[,,1]
	Xfdjf_fit["2100",,"CAMS-CSM1-0"] = 2 * Xfdjf_fit["2099",,"CAMS-CSM1-0"] - Xfdjf_fit["2098",,"CAMS-CSM1-0"]

# Trick to keep ny years (and not rewrite all scripts)
Xfdjf_fit["1850",,] = Xfdjf_fit["1851",,]

	France T, summer and winter: Xfjja_fit, Xfdjf_fit

		ACCESS-CM2

		ACCESS-ESM1-5

		AWI-CM-1-1-MR

		CanESM5-CanOE

		CanESM5

		CESM2

		CESM2-WACCM

		CMCC-CM2-SR5

		CNRM-CM6-1-HR

		CNRM-CM6-1

		CNRM-ESM2-1

		EC-Earth3-Veg

		FGOALS-f3-L

		FGOALS-g3

		GISS-E2-1-G

		INM-CM4-8

		IPSL-CM6A-LR

		MIROC6

		MIROC-ES2L

		MPI-ESM1-2-HR

		MPI-ESM1-2-LR

		MRI-ESM2-0

		NorESM2-LM

		NorESM2-MM

		TaiESM1

		UKESM1-0-LL

		ACCESS-CM2

		ACCESS-ESM1-5

		AWI-CM-1-1-MR

		CanESM5-CanOE

		CanESM5

		CESM2

		CESM2-WACCM

		CMCC-CM2-SR5

		CNRM-CM6-1-HR

		CNRM-CM6-1

		CNRM-ESM2-1

		EC-Earth3-Veg

		FGOALS-f3-L

		FGOALS-g3

		GISS-E2-1-G

		INM-CM4-8

		IPSL-CM6A-LR

		MIROC6

		MIROC-ES2L

		MPI-ESM1-2-HR

		MPI-ESM1-2-LR

		MRI-ESM2-0

		NorESM2-LM

		NorESM2-MM

		TaiESM1

		UKESM1-0-LL

		CAMS-CSM1-0

		CAMS-CSM1-0

		CAMS-CSM1-0

		CAMS-CSM1-0



## 4. Estimating GHG response

This is the most difficult step. The strategy is as follows. 

For GSAT (see details in Ribes et al., 2021, https://www.science.org/doi/10.1126/sciadv.abc0671):
- a. Some CMIP6 models have contributed to DAMIP and provided (complete) histghg runs. For these models, we estimate the GHG response by smoothing the GSAT from histghg runs.
- b. Many CMIP6 models simply did not contribute to DAMIP (meaning no histghg run available). For these, we reconstruct the missing histghg runs from the 1%CO2 experiment, and using the ensemble of DAMIP models as a learning sample (i.e. to learn the relationship between the histghg response and the 1%CO2 response). 

With start with a.

In [11]:
message("Forced GHG response in histghg runs (GSAT only)")
year_ghg = 1850:2020
ny_ghg = length(year_ghg)
is_histghg_full = (apply(is.na(Xg[as.character(year_ghg),"histghg",]),2,sum)==0)
Models_histghg_full = Models[is_histghg_full]
HatM_ghg_g = hatm(dfg["histghg"],year_ghg)
for (mod in Models_histghg_full) {
    Xg_fit[as.character(year_ghg),"ghg",mod] = HatM_ghg_g %*% Xg[as.character(year_ghg),"histghg",mod]
}

Forced GHG response in histghg runs (GSAT only)



Then b. Reconstruction of missing histghg runs

This is done using an adaptation of the KCC method. DAMIP models provide a sample of (1%CO2 + histghg) responses. Non-DAMIP models provide only the 1%CO2 run. So, we use DAMIP models to derive a prior on the (1%CO2+histghg) responses (including how these 2 things are related). Then, say for model M (a non-DAMIP model), we compute the posterior of its histghg response given its 1%CO2 "observations".

We first assess the 1%CO2 response (for all models)

In [12]:
message("Forced CO2 response in 1%-CO2 runs (GSAT only)")
year_pct = 1850:1999
ny_pct = length(year_pct)
HatM_pct_g = hatm(dfg["1pctCO2"],year_pct)
Xg_fit_pct = array(NA,dim=c(ny_pct,Nmod),dimnames=list(year=year_pct,model=Models))
for (mod in Models) {
    Xg_fit_pct[as.character(year_pct),mod] = smooth.spline(year_pct,Xg[as.character(year_pct),"1pctCO2",mod],df=dfg["1pctCO2"])$y
}

Forced CO2 response in 1%-CO2 runs (GSAT only)



In [13]:
message("Reconstructing non-available histghg runs")
Models_histghg = Models[ is_histghg_full ]

Sr = abind(Xg_fit_pct[, Models_histghg],#
           Xg_fit[as.character(year_ghg),"ghg", Models_histghg],#
           along=1,use.dnns=T)
#dimnames(Sr)$year = c(paste0(as.character(year_pct),"_pct"),paste0(as.character(year_ghg),"_ghg"))
Sr_mean = apply(Sr,1,mean)
Sr_var = var(t(Sr))

# Reconstruction
Models_histghg_recons = Models[!is_histghg_full]

for (mod in Models_histghg_recons) {
    # Extraction operator
    H_tmp = abind(diag(rep(1,ny_pct)),array(0,dim=c(ny_pct,ny_ghg)),along=2)
    sigma2_tmp = var(Xg[1:140,"1pctCO2",mod] - Xg_fit_pct[1:140,mod])
    Sigma_y_tmp = HatM_pct_g %*% diag( sigma2_tmp*ones(year_pct) ) %*% t(HatM_pct_g)
    v_recons = kriging(Sr_mean,Sr_var,Xg_fit_pct[as.character(year_pct),mod],Sigma_y_tmp,H_tmp)
    Xg_fit[as.character(year_ghg),"ghg",mod] = v_recons$mean[-(1:ny_pct)]
}

Reconstructing non-available histghg runs



For France temperature, estimation from histghg runs (where available) was challenging due to a higher internal variability at the regional level. Therefore, we rather assume patern scaling, and estimate the France GHG response as a rescaled GSAT GHG response. The rescaling coefficient is derived from 1%CO2 experiments (which all models have run). 

Again we start by estimating the forced response in 1%CO2 runs.

In [14]:
message("Forced CO2 response in 1%-CO2 runs (France T)")
HatM_pct_f = hatm(dff["1pctCO2"],year_pct)
Xf_fit_pct = array(NA,dim=c(ny_pct,Nmod),dimnames=list(year=year_pct,model=Models))
for (mod in Models) {
    Xf_fit_pct[as.character(year_pct),mod] = smooth.spline(year_pct,Xf[as.character(year_pct),"1pctCO2",mod],df=dff["1pctCO2"])$y
}

Forced CO2 response in 1%-CO2 runs (France T)



Then we apply pattern scaling to derive the GHG-induced warming over France.

In [15]:
dTg_1pct = apply(Xg_fit_pct[121:140,] - Xg_fit_pct[1:20,],2,mean)
dTf_1pct = apply(Xf_fit_pct[121:140,] - Xf_fit_pct[1:20,],2,mean)
Ratio_1pct = dTf_1pct / dTg_1pct
Xf_fit[,"ghg",] = (Xg_fit[,"ghg",] - ones(year)%o%apply(Xg_fit[1:50,"all",],2,mean)) * (ones(year) %o% Ratio_1pct) + ones(year)%o%apply(Xf_fit[1:50,"all",],2,mean)

The end:
- Xg_fit and Xf_fit are fullfilled.
- Same for Xfjja_fit and Xfdjf_fit (except the GHG response, which we do not consider).